In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from werkzeug.security import generate_password_hash, check_password_hash
import getpass
warnings.filterwarnings('ignore')

### Ingresamos con usuario y clave

In [2]:
#import webDriver
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')        #remove this for easy debbuing on your laptop /pc
#chrome_options.add_argument('--no-sandbox')                             
#chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('./chromedriver.exe',chrome_options=chrome_options)
driver.get("https://vi.grupo-penaflor.com.ar/")
user="mariano.parada@gmail.com"
password = getpass.getpass('Ingrese contraseña:')
driver.find_element_by_xpath('//*[@id="email"]').send_keys(user)
driver.find_element_by_xpath('//*[@id="pass"]').send_keys(password)
driver.find_element_by_xpath('//*[@id="send2"]').click()

Ingrese contraseña: ··········


### Recorremos las paginas y guardamos todo en un dataframe

In [3]:
bodegas=[]
imagenes=[]
precios=[]
precios_dto=[]
varietales=[]
vinos=[]

for i in range(1,12):
    driver.get('https://vi.grupo-penaflor.com.ar/vinos/tintos.html?p='+ str(i))
    html_code = driver.page_source
    soup = BeautifulSoup(html_code, 'lxml')
    elements = soup.find_all("li", class_="item product product-item")

    for element in elements:
        try:
            imagen = element.find("img")['src']
            bodega = element.find(class_="bodega").text
            varietal = element.find(class_="varietal").text
            vino = element.find(class_="product-item-link").text
            precio = element.find(class_="price").text
            price = element.find_all("span", class_="price")
            precio_dto = price[1].text
            #Agrego al info a cada lista para luego armar el DataFrame
            bodegas.append(bodega)
            imagenes.append(imagen)
            precios.append(precio)
            precios_dto.append(precio_dto)
            varietales.append(varietal)
            vinos.append(vino)
        except:
            NoSuchElementException
    time.sleep(1)

#Armo el DataFrame
peñaflor = pd.DataFrame({'bodega': bodegas, 'vino': vinos, 'varietal': varietales ,'precio': precios, 'precio_dto': precios_dto, 'foto': imagenes})#,'textos': textos })

#Formateamos los precios como número para luego calcular el descuento
peñaflor['precio'] = peñaflor['precio'].str.replace("$","")
peñaflor['precio'] = peñaflor['precio'].str.replace(".","")
peñaflor['precio'] = peñaflor['precio'].str.replace(",",".")
peñaflor['precio'] = peñaflor['precio'].astype(float)

peñaflor['precio_dto'] = peñaflor['precio_dto'].str.replace("$","")
peñaflor['precio_dto'] = peñaflor['precio_dto'].str.replace(".","")
peñaflor['precio_dto'] = peñaflor['precio_dto'].str.replace(",",".")
peñaflor['precio_dto'] = peñaflor['precio_dto'].astype(float)

peñaflor['descuento'] = 100-((peñaflor['precio_dto']*100)/peñaflor['precio'])
peñaflor['descuento'] = round(peñaflor['descuento']).astype(int)

driver.quit()

In [9]:
peñaflor.sort_values(by="descuento", ascending=False).head()

,bodega,vino,varietal,precio,precio_dto,foto,descuento
80,Finca Las Moras,Los Intocables,Malbec,1476.96,664.63,https://vi.grupo-penaflor.com.ar/media/catalog...,55
86,Trapiche,Fond de Cave Reserva,Cabernet Sauvignon,1639.89,737.95,https://vi.grupo-penaflor.com.ar/media/catalog...,55
60,Navarro Correas,Navarro Correas Reserva,Malbec,1639.89,901.93,https://vi.grupo-penaflor.com.ar/media/catalog...,45
17,El Esteco,Don David Reserva,Malbec,1639.89,901.93,https://vi.grupo-penaflor.com.ar/media/catalog...,45
40,Navarro Correas,Selección del Enólogo Grand Assemblage 2015,Blend,4120.36,2266.20,https://vi.grupo-penaflor.com.ar/media/catalog...,45


In [10]:
peñaflor.varietal.value_counts().head()

Malbec                41
Cabernet Sauvignon    29
Blend                 23
Syrah                  9
Pinot Noir             6
Name: varietal, dtype: int64

In [11]:
peñaflor.bodega.value_counts().head()

Trapiche             42
Finca Las Moras      25
El Esteco            24
Navarro Correas      16
Mascota Vineyards     6
Name: bodega, dtype: int64

In [7]:
peñaflor.precio.describe()

count      121.000000
mean      3120.557603
std       4540.843072
min        378.730000
25%       1123.590000
50%       1639.890000
75%       3160.020000
max      38006.160000
Name: precio, dtype: float64

In [21]:
#limpiar outliers 
porcentaje_anomalia = 25 # un % de 10 es equivalente a eliminar lo menor al percentil 10 y lo mayor a percentil 90
q3 = (100-porcentaje_anomalia)/100
q1 = (0+porcentaje_anomalia)/100
entrada = len(peñaflor.index)
df_sin_anomalos = peñaflor # Vamos a guardarlo en otro DataFrame para no pisar el anterior
columns = ['precio'] # Nombre de las columnas a revisar, si es más de una , separar por comas 'col1','col2'...
for i in columns:
    q1 = df_sin_anomalos[i].quantile(q1)
    q3 = df_sin_anomalos[i].quantile(q3)
    iqr = q3 - q1
    df_sin_anomalos = df_sin_anomalos[~((df_sin_anomalos[i] > q3))]
    print("límite inferior: ",df_sin_anomalos.precio.min())
    print("límite superior: ",q3 )
salida = len(df_sin_anomalos.index)
print("Se limpinaron: " , entrada-salida , " registros")
peñaflor = df_sin_anomalos

límite inferior:  378.73
límite superior:  3160.02
Se limpinaron:  30  registros


In [22]:
#Vemos cambios luego de quitar el 25% de los vinos más caros
peñaflor.precio.describe()

count      91.000000
mean     1470.483736
std       707.874981
min       378.730000
25%       897.160000
50%      1212.460000
75%      1639.890000
max      3160.020000
Name: precio, dtype: float64

In [36]:
# Eliminamos vinos de 375 y 500 ml
peñaflor = peñaflor[~peñaflor["vino"].str.contains('500')]
peñaflor = peñaflor[~peñaflor["vino"].str.contains('375')]

# Mostramos los vinos que valen menos de 1000 pesos ordenados por los de mayor precio con descuento
peñaflor.loc[(peñaflor['precio_dto']>600) & (peñaflor['precio_dto']<1000)].sort_values(by="precio_dto").head(50)

,bodega,vino,varietal,precio,precio_dto,foto,descuento
94,Finca Las Moras,Alma Mora Select Reserve,Cabernet Sauvignon,1011.44,606.86,https://vi.grupo-penaflor.com.ar/media/catalog...,40
4,Trapiche,Fond de Cave,Cabernet Sauvignon,1123.59,617.97,https://vi.grupo-penaflor.com.ar/media/catalog...,45
11,El Esteco,Don David,Malbec,1123.59,617.97,https://vi.grupo-penaflor.com.ar/media/catalog...,45
80,Finca Las Moras,Los Intocables,Malbec,1476.96,664.63,https://vi.grupo-penaflor.com.ar/media/catalog...,55
13,Trapiche,Trapiche Reserva,Merlot,1212.46,666.86,https://vi.grupo-penaflor.com.ar/media/catalog...,45
48,Trapiche,Trapiche Reserva,Malbec,1212.46,666.86,https://vi.grupo-penaflor.com.ar/media/catalog...,45
53,Trapiche,Fond de Cave,Malbec,1123.59,674.15,https://vi.grupo-penaflor.com.ar/media/catalog...,40
72,El Esteco,Don David,Syrah,1123.59,674.15,https://vi.grupo-penaflor.com.ar/media/catalog...,40
76,El Esteco,Don David,Tannat,1123.59,674.15,https://vi.grupo-penaflor.com.ar/media/catalog...,40
51,Navarro Correas,Coleccion Privada,Pinot Noir,1123.59,674.15,https://vi.grupo-penaflor.com.ar/media/catalog...,40
